In [30]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sympy as sp
from scipy.integrate import simpson, romberg, trapezoid
from scipy.special import roots_legendre

# Actividad 07: Integración

---
### Profesor: Juan Marcos Marín
### Nombre: Juan Josè Usuga Giraldo 1020482080
*Métodos computacionales 2024-II*

---

#1
* Implemente una función para el **método de integración de Romberg** definiendo un límite de tolerancia de 1e-8 y/o un máximo de iteraciones de 10.

* Encuentre la integral para

$$\int_0^{\pi/4} dx\, e^{3x}\cdot \sin(x)$$

* Imprima su resultado y compare los valores dados por `scipy.integrate.romberg`

* Finalmente, encuentre el valor del error, hallando el valor exacto usando `sympy`



In [8]:
def JRomberg(f, a, b, eps = 1e-8, max_iter=10):
  R = np.zeros((max_iter, max_iter))
  n = 1
  h = b - a
  R[0, 0] = (h / 2) * (f(a) + f(b))
  for i in range(1, max_iter):
    n *= 2
    h /= 2
    suma = sum(f(a + (k - 0.5) * h) for k in range(1, n, 2))
    R[i, 0] = 0.5 * R[i-1, 0] + h * suma
    for j in range(1, i + 1):
      R[i, j] = R[i, j-1] + (R[i, j-1] - R[i-1, j-1]) / (4**j- 1)
    if abs(R[i, i] - R[i-1, i-1]) < eps:
      return R[i, i], R[:i+1, :i+1]
  return R[i, i], R[:i+1, :i+1]

In [9]:
f = lambda x: np.exp(3*x)*np.sin(x)
eps = 1e-8
i = JRomberg(f,0,np.pi/4,eps)
i=i[0]
print(i)

1.576039519844102


In [12]:
a, b = 0, np.pi/4
i_scipy = romberg(f, a, b)
print(f'Integral mediante scipy.integrate.romberg:', i_scipy.round(3))

Integral mediante scipy.integrate.romberg: 1.592


<ipython-input-12-c4a96ac305e6>:2: DeprecationWarning: `scipy.integrate.romberg` is deprecated as of SciPy 1.12.0and will be removed in SciPy 1.15.0. Please use`scipy.integrate.quad` instead.
  i_scipy = romberg(f, a, b)


In [13]:
error_romberg = abs((i - i_scipy)/i)
print(f'Error relativo:', error_romberg)

Error relativo: 0.010188950094410966


In [16]:
x = sp.Symbol('x')
i_sympy = sp.integrate(sp.exp(3*x)*sp.sin(x), (x, 0, np.pi/4))
print(f'Integral mediante sympy:', i_sympy.round(3))
error_sympy = abs((i - i_sympy)/i)
print(f'Error relativo:', error_sympy)

Integral mediante sympy: 1.592
Error relativo: 0.0101889500944530


#2

* Usando los *métodos trapezoidal compuesto*, *simpson 1/3* y de *medio punto* encuentre la siguiente integral,

$$\int_e^{1+e} dx\, \frac{1}{x\ln x}$$

* Luego, haga un estudio de la convergencia en términos del valor de $h$ o de los sub-intervalos de la función. ¿Cuál es mejor?


In [17]:
f = lambda x: 1/(x*np.log(x))
a, b = np.e, 1+np.e

In [22]:
# Trap Compuesto
nx = 10
x = np.linspace(a, b, nx+1)
y = f(x)
Z = trapezoid(y, x)
print(f'Integral de la función mediante trapezoidal compuesto:', Z.round(5))

Integral de la función mediante trapezoidal compuesto: 0.27266


In [23]:
def find_h(f, a, b, tol, metodo_de_integracion):
  n = 2
  h = (b-a)/n
  x = np.linspace(a, b, n+1)
  y = f(x)
  integral = 0
  while True:
    i_new = metodo_de_integracion(y, x)
    if abs(integral-i_new) < tol:
      return i_new, n, h
    n *=2
    h = (b-a)/n
    x = np.linspace(a, b, n+1)
    y = f(x)
    integral = i_new
    return i_new, n, h

In [25]:
ih_trapezoid, n , h = find_h(f,a,b, 1e-5, trapezoid)
print(f'Integral trapezoidal compuesto:', ih_trapezoid.round(4) , 'con h =', h)

Integral trapezoidal compuesto: 0.2761 con h = 0.25


In [27]:
# Simpson 1/3
def simpson13(f, a, b, n): # Tomado de los apuntes de clase
  h = (b - a) / n
  integral = (f(a) + f(b))
  # Suma de los términos impares
  for i in range(1, n, 2):
    x_i = a + i * h
    integral += 4 * f(x_i)
  # Suma de los términos pares
  for i in range(2, n, 2):
    x_i = a + i * h
    integral += 2 * f(x_i)
  return integral * h/3

In [28]:
n_simp = 12  # número de áreas a sumar (debe ser par para Simpson).

result = simpson13(f, a, b, n)
h_simpson = (b-a)/n
print("Integral por Simpson 1/3", result.round(3), 'con h=', h_simpson)

Integral por Simpson 1/3 0.273 con h= 0.25


In [29]:
# punto medio
def punto_medio(f,a,b,n):
  h = (b-a)/n
  r = 0
  for i in range(0, n):
    x_i = a + i * h
    r += f(x_i + h/2)
  return h * r
print(f"Integral por punto medio {punto_medio(f, a, b, 10):.3f}")

Integral por punto medio 0.272


#3
Usando la siguiente función:



```python
def gauss_quad_standard(func, n):
    """
    Calcula la integral de una función en el intervalo [-1, 1]
    utilizando cuadratura gaussiana.

    Parameters:
    - func: La función a integrar.
    - n: Número de puntos para la cuadratura (grado del polinomio de Legendre).

    Returns:
    - Aproximación de la integral.
    """
    # Obtener raíces y pesos del polinomio de Legendre
    x, w = roots_legendre(n)

    # Evaluar la suma ponderada
    integral = np.sum(w * func(x))
    return integral
```

Modifique la función `gauss_quad_standard` de forma tal que no este restringida para $[-1,1]$ sino para cualquier intervalo $[a,b]$. Luego, encuentre la integral del *punto 2*.





In [31]:
def gauss_quad_standard(func, a, b, n):
    import sympy as sp
    x = sp.Symbol('x')
    # Obtener raíces y pesos del polinomio de Legendre
    xi, w = roots_legendre(n)
    func_ = func(np.abs((b-a)/2*x + (a+b)/2))
    f = sp.lambdify(x, func_)
    # Evaluar la suma ponderada
    integral = (b-a)/2 * np.sum(w * f(xi))
    return integral

In [32]:
f3 = lambda x: 1/(x*sp.log(x))
gauss_quad_standard(f3, np.e, np.e+1, 4)

0.27251386118132886

#4

Encuentra todas las raices para los polinomios de grado 3 y 4 de **Legendre** usando el Método de la Secante y Newton-Raphson.



```python
import sympy as sp
x = sp.Symbol('x')

# Polinomio de Legendre de grado n
Pn = sp.legendre(n, x)

```

y calcule los pesos $w_i$ de la cuadratura mediante la fórmula:
   $$
   w_i = \frac{2}{(1 - x_i^2) \left[P_n'(x_i)\right]^2},
   $$
   donde $P_n'(x)$ es la derivada del polinomio de Legendre $P_n(x)$.


In [35]:
x = sp.Symbol('x')
# Pol de Legendre de grado n
P3 = sp.legendre(3, x)
dP3 = sp.diff(P3, x, 1)
P4 = sp.legendre(4, x)
dP4 = sp.diff(P4, x, 1)
print('Polinomios de Legendre grado 3 y 4 con sus respectivas derivadas:\n\n')
display(P3, dP3, P4, dP4)
print("valor numerico de las raices")
np.sqrt(15)/5

Polinomios de Legendre grado 3 y 4 con sus respectivas derivadas:




5*x**3/2 - 3*x/2

15*x**2/2 - 3/2

35*x**4/8 - 15*x**2/4 + 3/8

35*x**3/2 - 15*x/2

valor numerico de las raices


0.7745966692414834

In [36]:
P3_func = sp.lambdify(x, P3, 'numpy')
dP3_func = sp.lambdify(x, dP3, 'numpy')

P4_func = sp.lambdify(x, P4, 'numpy')
dP4_func = sp.lambdify(x, dP4, 'numpy')

In [37]:
# Raíces por Newton, n = 3
from scipy.optimize import newton
root3_n1 = newton(P3_func, 0, dP3_func)
root3_n2 = newton(P3_func, 1, dP3_func)
root3_n3 = newton(P3_func, -1, dP3_func)
lista_roots_P3 = [root3_n1, root3_n2, root3_n3]
lista_pesos3 = []
for i in lista_roots_P3:
  wi = 2 / ((1 - i**2)*(dP3_func(i))**2)
  lista_pesos3.append(wi)
print('Raíces por Newton del polinomio de Legendre grado 3:\n', lista_roots_P3)
print('\n\nPesos de Newton del polinomio de Legendre grado 3:\n', lista_pesos3)

Raíces por Newton del polinomio de Legendre grado 3:
 [0.0, 0.7745966692414835, -0.7745966692414835]


Pesos de Newton del polinomio de Legendre grado 3:
 [0.8888888888888888, 0.5555555555555552, 0.5555555555555552]


In [39]:
# Raíces por Secante para Polinomio n = 4
root4_n1 = newton(P4_func, -0.2, dP4_func)
root4_n2 = newton(P4_func, 1, dP4_func)
root4_n3 = newton(P4_func, -1, dP4_func)
root4_n4 = newton(P4_func, 0.5, dP4_func)
lista_roots_P4 = [root4_n1, root4_n2, root4_n3, root4_n4]
lista_pesos4 = []
for i in lista_roots_P4:
  wi = 2 / ((1 - i**2)*(dP4_func(i))**2)
  lista_pesos4.append(wi)
print('Raíces por Secante del polinomio de Legendre grado 4:\n\n', lista_roots_P4)
print('\n\nPesos de Secante del polinomio de Legendre grado 4:\n\n', lista_pesos4)

Raíces por Secante del polinomio de Legendre grado 4:

 [-0.3399810435848563, 0.861136311594053, -0.861136311594053, 0.33998104358485626]


Pesos de Secante del polinomio de Legendre grado 4:

 [0.6521451548625463, 0.3478548451374528, 0.3478548451374528, 0.6521451548625462]
